### Mount Azure Data Lake using Service Principal
1. Get client_id, tenant_id and client_secret from key vault
2. Set spark config with app / client id, directory / tenant id & secret
3. Call file system utility mount to mount the storage
4. Explore other file system utilities related to mount (list all mounts, unmounts)

In [0]:
def mount_adls(storage_account_name, container_name):
    #Get Secret Keys from Vault
    client_id = dbutils.secrets.get(scope='DatabricksKeyVaultX', key='ClientID')
    tenant_id = dbutils.secrets.get(scope='DatabricksKeyVaultX',key='TenantID')
    client_secret = dbutils.secrets.get(scope='DatabricksKeyVaultX',key='ClientSecret')

    #Set Spark Configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
                "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
                "fs.azure.account.oauth2.client.id": client_id,
                "fs.azure.account.oauth2.client.secret": client_secret,
                "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    #Check if mount exists, if Yes then unmount it
    if any(mount.mountPoint ==f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    #Mount the storage account container
    dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account_name}/{container_name}",
    extra_configs = configs)

    # Display the list of mounts
    display(dbutils.fs.mounts())

In [0]:
for i in ['raw', 'presentation', 'processed']:
    mount_adls('rupertsy1dl', i)

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/rupertsy1dl/demo,abfss://demo@rupertsy1dl.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/mnt/rupertsy1dl/raw,abfss://raw@rupertsy1dl.dfs.core.windows.net/,
/,DatabricksRoot,


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/rupertsy1dl/demo,abfss://demo@rupertsy1dl.dfs.core.windows.net/,
/mnt/rupertsy1dl/presentation,abfss://presentation@rupertsy1dl.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/mnt/rupertsy1dl/raw,abfss://raw@rupertsy1dl.dfs.core.windows.net/,


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/rupertsy1dl/demo,abfss://demo@rupertsy1dl.dfs.core.windows.net/,
/mnt/rupertsy1dl/presentation,abfss://presentation@rupertsy1dl.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/mnt/rupertsy1dl/raw,abfss://raw@rupertsy1dl.dfs.core.windows.net/,
